In [20]:
import py_entitymatching as em
import os, sys
import pandas as pd

In [21]:
#datasets directory
#datasets_dir = "/home/zeus/data_science/datasets/datasets"
datasets_dir = "/Users/nehamittal/Downloads/datasets"
#input tables
path_songs = datasets_dir + "/" +'songs.csv'
path_tracks= datasets_dir+"/"+"tracks.csv"

In [22]:
songs = em.read_csv_metadata(path_songs, key='id',low_memory=False)
tracks = em.read_csv_metadata(path_tracks, key='id',low_memory=False)
#schema of both songs and tracks- id,title,artist,year

In [137]:
#Downsample
sample_songs,sample_tracks=em.down_sample(songs,tracks,size=3000,y_param=6,show_progress=False)
#len(sample_songs),len(sample_tracks)


(16029, 3000)

In [139]:
block_f = em.get_features_for_blocking(sample_songs, sample_tracks)
#block_f

In [143]:
ob = em.OverlapBlocker()
new_stop_word_list=["In","A","And","Are","As","At","Be","By","For","From","Has","He","In","Is","It","Its","On","That","The","To","Was","Were","Will","With","In","Theme","theme"]
for new_stop_word in new_stop_word_list:
    ob.stop_words.append(new_stop_word)

In [144]:
tuples_set = ob.block_tables(sample_songs, sample_tracks, 'title','title',
                    word_level=True, overlap_size=1,
                    allow_missing=True,
                    rem_stop_words=True,
                    l_output_attrs=['title', 'artist', 'year'], 
                    r_output_attrs=['title', 'artist', 'year'],
                    show_progress=False)
tuples_set=ob.block_candset(tuples_set,'artist','artist',
                            rem_stop_words=True,
                            word_level=True,
                            overlap_size=1,
                            show_progress=False)

0%                          100%
[##############################] | ETA: 00:00:17 | ETA: 00:00:15 | ETA: 00:00:15 | ETA: 00:00:14 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:13 | ETA: 00:00:12 | ETA: 00:00:12 | ETA: 00:00:11 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:17


In [145]:
block_f=em.get_features_for_blocking(sample_songs,sample_tracks)
#block_f

In [146]:
rb = em.RuleBasedBlocker()
rb.add_rule(['title_title_lev_dist(ltuple, rtuple) > 15'],block_f)
rb.add_rule(['artist_artist_lev_dist(ltuple, rtuple) > 15'],block_f)
tuples_set=rb.block_candset(tuples_set)
tuples_set.to_csv(datasets_dir + "/" +"tuples_set_after_blocking.csv", encoding='utf-8')

0%                          100%
[                              ]0%                          100%
[##############################] | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


In [147]:
S = em.sample_table(tuples_set, 120)
S.to_csv(datasets_dir + "/" +"sampled.csv", encoding='utf-8')